<H1>DocCalendar Fuzzer</H1>

In [ ]:
from fuzzingbook import WebFuzzer
from fuzzingbook.GrammarFuzzer import GrammarFuzzer
from fuzzingbook.Grammars import crange, is_valid_grammar, syntax_diagram, Grammar
import requests
from urllib.parse import urljoin, urlsplit
from http.server import HTTPStatus

In [ ]:
from random import randrange
from datetime import timedelta, datetime

def random_date():
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    start = datetime.strptime('1/1/2000 1:00 AM', '%m/%d/%Y %I:%M %p')
    end = datetime.strptime('1/1/2024 11:59 PM', '%m/%d/%Y %I:%M %p')

    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    result = start + timedelta(seconds=random_second)
    return datetime.strftime(result, '%m/%d/%Y %H:%M:%S')


<H3>Generate the grammar </H3>

In [ ]:
ORDER_GRAMMAR : Grammar = {
    "<start>" : ["<patient/appointments>"],
    "<patient/appointments>" : ["/patient/appointments?p_id=<p_id>&datetime='<datetime>'"],
    "<p_id>" : crange('1', '5'),
    "<datetime>" : [random_date() for i in range(150)]

}

assert is_valid_grammar(ORDER_GRAMMAR)

order_fuzzer = GrammarFuzzer(ORDER_GRAMMAR)
[order_fuzzer.fuzz() for i in range(100)]




In [ ]:
while True:
    path = order_fuzzer.fuzz()
    source = 'http://127.0.0.1:5000'
    url = urljoin(source, path)
    r = requests.get(url)
    if r.status_code == HTTPStatus.INTERNAL_SERVER_ERROR:
        break

<H3>Mutation Fuzzer</H3>

In [ ]:
from fuzzingbook.MutationFuzzer import MutationFuzzer



mutate_order_fuzzer = MutationFuzzer([order_fuzzer.fuzz()], min_mutations=1, max_mutations=1)
[mutate_order_fuzzer.fuzz() for i in range(100)]


In [9]:
while True:
    path = mutate_order_fuzzer.fuzz()
    source = 'http://127.0.0.1:5000'
    url = urljoin(source, path)
    r = requests.get(url)
    if r.status_code == HTTPStatus.INTERNAL_SERVER_ERROR:
        break

url

"http://127.0.0.1:5000/patient/appointments?p_id=X5&datetime='07/27/2012 22:20:03'"

In [10]:
failing_path = path
failing_path


"/patient/appointments?p_id=X5&datetime='07/27/2012 22:20:03'"

In [13]:
from fuzzingbook.Fuzzer import Runner

class WebRunner(Runner):
    """Runner for a Web server"""

    def __init__(self, base_url):
        self.base_url = base_url

    def run(self, url: str):
        if self.base_url is not None:
            url = urljoin(self.base_url, url)

        import requests  # for imports
        r = requests.get(url)
        if r.status_code == HTTPStatus.OK:
            return url, Runner.PASS
        elif r.status_code == HTTPStatus.INTERNAL_SERVER_ERROR:
            return url, Runner.FAIL
        else:
            return url, Runner.UNRESOLVED
        


In [14]:
web_runner = WebRunner(source)
web_runner.run(failing_path)



("http://127.0.0.1:5000/patient/appointments?p_id=X5&datetime='07/27/2012 22:20:03'",
 'FAIL')

In [15]:
from fuzzingbook.Reducer import DeltaDebuggingReducer 


minimized_path = DeltaDebuggingReducer(web_runner).reduce(failing_path)
minimized_path

'patient/appointments?p_id'